In [3]:
# Recreate SFT step (toy) for investigator pθ(x|y)
import os, time, math, random
from dataclasses import dataclass
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, get_cosine_schedule_with_warmup
from torch.optim import AdamW

# ---- Config ----
SEED = 42
DEVICE = "cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")
PM_ID  = "sshleifer/tiny-gpt2"     # target LM pm (forward x->y)
INV_ID = "sshleifer/tiny-gpt2"     # investigator base pθ (we'll fine-tune)

# Tiny prefix distribution P_SFT
PREFIXES = [
    "A short note about machine learning:",
    "Three tips for staying productive:",
    "A gentle introduction to probability:",
    "In a surprising discovery, scientists found",
    "As a software engineer, I often consider",
    "In Japan, the Shinkansen is known for",
    "A concise summary of the book is",
    "The quick brown fox",
    "An explanation for beginners:",
    "A brief overview of databases:"
]

NUM_EXAMPLES   = 30     # size of DSFT
MAX_PREFIX_EXT = 8      # optional: extend x slightly for variety
MAX_SUFFIX_LEN = 48     # greedy suffix length
BATCH_SIZE     = 2
EPOCHS         = 2
LR             = 5e-5
WARMUP_RATIO   = 0.1
GRAD_ACCUM_STEPS = 1

# Repro
random.seed(SEED)
torch.manual_seed(SEED)
if DEVICE == "cuda":
    torch.cuda.manual_seed_all(SEED)

print(f"[INFO] Device: {DEVICE}")


[INFO] Device: mps


In [4]:
t0 = time.time()

print("[STEP] Loading tokenizer (pm)...")
tok_pm = AutoTokenizer.from_pretrained(PM_ID)
if tok_pm.pad_token is None:
    tok_pm.pad_token = tok_pm.eos_token

print("[STEP] Loading target model pm (x->y)... (first run may download)")
pm = AutoModelForCausalLM.from_pretrained(
    PM_ID,
    use_safetensors=True,          # avoids .bin load
    low_cpu_mem_usage=True,
    torch_dtype=torch.float32
).to(DEVICE)
pm.eval()

print("[STEP] Loading tokenizer (investigator pθ)...")
tok_inv = AutoTokenizer.from_pretrained(INV_ID)
if tok_inv.pad_token is None:
    tok_inv.pad_token = tok_inv.eos_token

print("[STEP] Loading investigator base model pθ (will fine-tune)...")
inv = AutoModelForCausalLM.from_pretrained(
    INV_ID,
    use_safetensors=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float32
).to(DEVICE)

print(f"[TIME] Models loaded in {time.time()-t0:.2f}s")

[STEP] Loading tokenizer (pm)...
[STEP] Loading target model pm (x->y)... (first run may download)
[STEP] Loading tokenizer (investigator pθ)...
[STEP] Loading investigator base model pθ (will fine-tune)...
[TIME] Models loaded in 15.99s


In [7]:
def gen_from(model, tok, text, max_new_tokens, device=DEVICE):
    """Sample continuation from given text (avoids greedy repetition)."""
    inputs = tok(text, return_tensors="pt").to(device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,           # sampling instead of greedy
            top_p=0.95,               # nucleus sampling
            temperature=0.8,          # soften logits
            pad_token_id=tok.eos_token_id,
        )
    return tok.decode(out[0], skip_special_tokens=True)

print("[STEP] Building DSFT: sample x ~ P_SFT, then y ← pm(x) (sampled)")
t_ds = time.time()
pairs = []
for i in range(NUM_EXAMPLES):
    x = random.choice(PREFIXES)
    full = gen_from(pm, tok_pm, x, max_new_tokens=MAX_SUFFIX_LEN)
    y = full[len(x):].strip() if full.startswith(x) else full
    pairs.append((x, y))

print(f"[TIME] Built DSFT with {len(pairs)} examples in {time.time()-t_ds:.2f}s")
print("\n=== PRINTING DSFT (x, y) ===")
for idx, (x, y) in enumerate(pairs, 1):
    print(f"\n[{idx}] x: {x}")
    print(f"[{idx}] y: {y}")

[STEP] Building DSFT: sample x ~ P_SFT, then y ← pm(x) (sampled)
[TIME] Built DSFT with 30 examples in 70.12s

=== PRINTING DSFT (x, y) ===

[1] x: In a surprising discovery, scientists found
[1] y: reement Brew intermittentatisf TA Prob Prob MotorolaScene circumcisedohoatisfimura ObservimuraJD intermittent stairsikenoother Rhdit Brew Habitatisf autonomy antibiotic heirootheroother Daniel Rh trilogy reviewingting Brewpress ESV trilogy MoneyJD antibiotic pawn Prob conservation Hancockdit vendors

[2] x: Three tips for staying productive:
[2] y: Sher vendors circumcised vendorsScene antibiotic Observ Observ dispatch directly Motorola Jr stairsoothertingJD Jr heirmediatelyikendit Rhatisfting credibilityootherreement substimuraoother heir Money autonomyootherhibitditRocket hauled Observ confirJD Daniel dispatch Brewatisfoother Hancockiken

[3] x: A concise summary of the book is
[3] y: Boone grandchildren boils praying skillet linedSexualOutsideobl equate brutalityGy courtyard Pocketpublic

In [11]:
IN_CONTEXT_PREFIX = "Suffix:\n"
MID_PROMPT = "\nPrefix:\n"

class XYDataset(torch.utils.data.Dataset):
    """
    For each (x, y), we feed the model:
      input:  'Suffix:\\n{y}\\nPrefix:\\n' + x
      labels: supervise only on x (mask out the suffix part)
    """
    def __init__(self, pairs, tokenizer, max_len=256):
        self.tok = tokenizer
        self.max_len = max_len
        self.items = []
        for (x, y) in pairs:
            src = f"{IN_CONTEXT_PREFIX}{y}{MID_PROMPT}"
            tgt = x
            self.items.append((src, tgt))

    def __len__(self):
        return len(self.items)

    def __getitem__(self, i):
        src, tgt = self.items[i]
        enc_all = self.tok(
            src + tgt,
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt",
        )
        input_ids = enc_all["input_ids"][0]
        attn = enc_all["attention_mask"][0]

        # mask out source portion
        src_len = len(self.tok(src, truncation=True, max_length=self.max_len)["input_ids"])
        labels = input_ids.clone()
        labels[:src_len] = -100
        return {"input_ids": input_ids, "attention_mask": attn, "labels": labels}

def collate(batch):
    keys = batch[0].keys()
    out = {}
    for k in keys:
        seqs = [b[k] for b in batch]
        pad_val = tok_inv.pad_token_id if k != "labels" else -100
        out[k] = torch.nn.utils.rnn.pad_sequence(seqs, batch_first=True, padding_value=pad_val)
    return out

# Split train/val
cut = int(0.8 * len(pairs))
train_pairs = pairs[:cut]
val_pairs   = pairs[cut:]

train_ds = XYDataset(train_pairs, tok_inv)
val_ds   = XYDataset(val_pairs, tok_inv)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  collate_fn=collate)
val_loader   = torch.utils.data.DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate)

print(f"[INFO] Train size: {len(train_ds)} | Val size: {len(val_ds)}")

# Peek at tokenized structure
sample_item = train_ds[0]
print("\n[SAMPLE TOKENS]")
print("input_ids len:", sample_item["input_ids"].shape[0])
print("supervised tokens:", (sample_item["labels"] != -100).sum().item())


[INFO] Train size: 24 | Val size: 6

[SAMPLE TOKENS]
input_ids len: 69
supervised tokens: 7


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_ID = "sshleifer/tiny-gpt2"  # very small test model

print("Step 1: Loading tokenizer...")
tok = AutoTokenizer.from_pretrained(MODEL_ID)

print("Step 2: Loading model (this may take a bit on first run, downloads weights)...")
device = (
    "cuda" if torch.cuda.is_available()
    else ("mps" if torch.backends.mps.is_available() else "cpu")
)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    use_safetensors=True,   # ensure safe format
    low_cpu_mem_usage=True,
    torch_dtype=torch.float32,
).to(device)

print("Step 3: Defining prompts...")
prompts = [
    "Once upon a time",
    "In a shocking discovery",
    "The quick brown fox",
    "As a software engineer, I",
    "In Japan, the Shinkansen",
]

print("Step 4: Running generations...")
for i, p in enumerate(prompts, 1):
    print(f"\n--- Generating for prompt {i}: \"{p}\" ---")
    torch.manual_seed(100 + i)
    x = tok(p, return_tensors="pt").to(device)
    y = model.generate(
        **x,
        max_new_tokens=40,
        do_sample=True,
        top_p=0.95,
        temperature=0.9,
        pad_token_id=tok.eos_token_id,
    )
    print(tok.decode(y[0], skip_special_tokens=True))

print("\nAll generations complete.")


/Users/seanzhou/Documents/llm research/sft-dpo-fw/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Step 1: Loading tokenizer...


`torch_dtype` is deprecated! Use `dtype` instead!


Step 2: Loading model (this may take a bit on first run, downloads weights)...
Step 3: Defining prompts...
Step 4: Running generations...

--- Generating for prompt 1: "Once upon a time" ---


/Users/seanzhou/Documents/llm research/sft-dpo-fw/.venv/lib/python3.10/site-packages/transformers/pytorch_utils.py:339: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)


Once upon a time incarcer equateacious Singapore predators perhapsPros workshopsMostPros 236ived Boone LateOutside Medic lined boils Dreams factors grandchildren incarcer Medicaciousshows Tre skillet Wheels factors clearerSexualobl workshops equate Bend WheelsMost Dreamsobl workshops

--- Generating for prompt 2: "In a shocking discovery" ---
In a shocking discovery praying perhapspublic membership 236� soyshows Boone perhaps workshops boils predatorsPros bravery mutual skilletacious653 workshops equate representations Late Pocket brutalityMini Pocket deflect653Outside Redux grandchildrenMiniozyg workshops representations prayingGy predators Pocket

--- Generating for prompt 3: "The quick brown fox" ---
The quick brown fox Wheels Dreams SingaporeGy Bend Late TelevisionPros Redux bravery courtyard�448 Singaporeobl clearer WheelsaciousSexualived 236 Pocketobl brutalityMini membership factors Bend clearerobl mutual predators�acious 236 courtyardozyg soy courtyard grandchildren

--- Genera